Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



In [1]:
import random as rd


def cria_cidades(n):
    """Cria um dicionário aleatório de cidades com suas posições (x,y).

    Argumentos
      n: inteiro positivo
        Número de cidades que serão visitadas pelo caixeiro.

    Retorno:
      Dicionário contendo o nome das cidades como chaves e a coordenada no plano
      cartesiano das cidades como valores.
    """

    cidades = {}

    for i in range(n):
        cidades[f"Cidade {i}"] = (rd.random(), rd.random())

    return cidades

## Importações



In [2]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



In [3]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 5
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [4]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

In [5]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness    

In [6]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [7]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)



Melhor individuo obtido por algoritmos genéticos:
['Cidade 3', 'Cidade 1', 'Cidade 0', 'Cidade 2', 'Cidade 4'] com distância: 1.9551939415093746

Melhor individuo obtido por busca exaustiva:
('Cidade 0', 'Cidade 3', 'Cidade 4', 'Cidade 2', 'Cidade 1') com distância: 1.9551939415093746


## Conclusão
<p style="text-align: justify"> O problema a ser resolvido consiste na descoberta da menor distância possível para o caixeiro viajante entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos. O caixeiro viajante é uma personagem que percorre de cidade a cidade a fim de vender os seus produtos! </p>
<p style="text-align: justify">  Este notebook nos permite identificar a versatilidade dos algoritímos genéticos, nos permitindo novamente, desenvolver problemas de minimização utilizando os conceitos genéticos discutidoss nos experimentos anteriores, pois se deseja resolver o problema observando a menor distância possível do viajante em cada cidade. Outro tópico importante para a resolução do problema é a respeito da limitação imposta, isto é, as condições de que o viajante não poderia passar mais de uma vez pela mesma cidade e  de que ele precisava retornar para a cidade inicial. </p>
<p style="text-align: justify"> Essas condições abordam um problema de difícil resolução, de caráter NP dificíl. Para quem não lembra,  NP (Non-Deterministic Polynomial time) é um conjunto de problemas que são decidíveis em tempo polinomial por uma máquina de Turing não-determinística. Logo, deve-se determinar todas as situações possíveis para se conseguir determinar qual é de fato a melhor solução, isto é, o menor caminho a ser percorrido pelo caixeiro entre as cidades. </p>
<p style="text-align: justify"> A estratégia utilizada para a resolução do desafio é a construção de funções, em especial, funções de cálculo de distância entre as cidades, a de mutação, cruzamento  e uma função de objetivo que vai solucionar o nosso problema. A primeira função se baseou na fórmula de cálculo euclidiano de dois pontos referentes a um ponto x e y, por sua vez, a função de cruzamento é de caráter de cruzamento ordenado .E por fim, desenvolveu-se a função objetivo que consistia de um loop que calculava as distâncias obtidas nos diferentes caminhos possíveis, retornando a menor distância possível. </p>
<p style="text-align: justify"> É interessante de se observar que ao se rodar o código diversas vezes, temos diferentes resultados minimos e isso porque? Justamente, porque o caminho do caixeiro pode se iniciar a cada n ponto, a cada n cidade e essa delimitação é determnada por meio da nossa função objetivo que retorna a distância percorrida pelo caixeiro seguindo o caminho contido no possível caminho de escolha do caixeiro. Lembrando que após percorrer todas as cidades em ordem, o caixeiro retorna para a cidade original de onde começou sua viagem. </p>

## Playground



In [8]:
print("Muito obrigada pela aula Dani :)")

Muito obrigada pela aula Dani :)
